# Spatially implicit simulations of early tetrapod diversity

Simulations are non-spatial (spatially implicit) so that a baseline of diversity can be achieved.

Simulations are dictated by three parameters: the speciation rate, the local density and the metacommunity size. The same speciation rate is used to generate both the local and the metacommunity.

In [1]:
import os
import pandas as pd

import numpy as np
from math import ceil
import random

In [2]:
# For dev use only - auto-reloading of modules
%load_ext autoreload
%aimport pycoalescence.coalescence_tree
%aimport pycoalescence.simulation
from pycoalescence.coalescence_tree import check_sql_table_exist
from pycoalescence import Simulation, CoalescenceTree
%autoreload 1

In [3]:
intervals = {'artinskian',
 'asselian',
 'bashkirian',
 'gzhelian',
 'kasimovian',
 'kungurian',
 'moscovian',
 'sakmarian'}

tetrapod_groups = {'amniote', 'amphibian'}

immigration_rates = 10.0**-np.linspace(0.1, 4, 20)
speciation_rates = [10**-x for x in range(1, 9)]

In [5]:
# The location to store simulations in
# output_dir = "/Volumes/Seagate 3TB/Paleo/Results/nse_01_single"
output_dir = "/run/media/sam/Media/Paleo/Results/nse_01_single"
if not os.path.exists(output_dir):
	os.makedirs(output_dir)
# The location of the csv containing location data for the fossil sites
csv_directory = "../MainSimulationR/input"
# The location to output the results csv to
output_csv_directory = "../../Results/nse1_single"
if not os.path.exists(output_csv_directory):
	os.makedirs(output_csv_directory)

In [7]:
# Read the data for each paleo coord
info_per_pcoord = pd.read_csv(os.path.join(csv_directory, "info_per_pcoord_main.csv"))
info_per_pcoord['lat'] = np.NaN
info_per_pcoord['long'] = np.NaN
# split into lat and long for shapefile
for index, row in info_per_pcoord.iterrows():
	lat, long = row["pcoords"].split(",")
	info_per_pcoord.loc[index, 'lat'] = pd.to_numeric(lat) # plus some small modifier\n",
	info_per_pcoord.loc[index, 'long'] = pd.to_numeric(long)
info_per_pcoord["interval"] = [x.lower() for x in info_per_pcoord["interval"]]
total_per_interval = info_per_pcoord.groupby(["tetrapod_group"],
										   squeeze=True)["individuals_total"].sum().reset_index()
local_community_size = max(total_per_interval.individuals_total)

In [8]:
total_per_interval

,tetrapod_group,individuals_total
0,amniote,533
1,amphibian,904


In [8]:
job_type = 0
from pycoalescence import Simulation, CoalescenceTree
for tet_group in tetrapod_groups:
	print(tet_group)
	job_type += 1
	simulated_number = total_per_interval[total_per_interval.tetrapod_group == tet_group].individuals_total
	for seed in range(1, 11, 1):
		sim = Simulation(logging_level=40)
		try:
			sim.set_simulation_parameters(seed=seed, job_type=job_type, output_directory=output_dir,
										  min_speciation_rate=0.0001, spatial=False, deme=int(simulated_number))
		except IOError:
			continue
		sim.set_speciation_rates(immigration_rates.tolist())
		try:
			sim.run()
		except IOError:
			pass
		tree = CoalescenceTree(sim, logging_level=40)
		# tree.wipe_data()
		tree.set_speciation_parameters(speciation_rates=immigration_rates)
		# tree.add_metacommunity_parameters(10000, 0.1, metacommunity_option="simulated")
		for metacommunity_size in [10**x for x in range(4, 9)]:
			for speciation_rate in speciation_rates:
				tree.add_metacommunity_parameters(metacommunity_size=metacommunity_size,
												  metacommunity_speciation_rate=speciation_rate,
												  metacommunity_option="analytical")
		tree.apply()

amphibian


amniote


In [9]:
# Now randomly select to get the species richness for each fragment
import sqlite3

from pycoalescence.sqlite_connection import check_sql_table_exist

job_type = 0
output_richness = []
output_alpha = []
output_fragment_richness = []
from pycoalescence import CoalescenceTree

for tet_group in tetrapod_groups:
	job_type += 1
	total_per_tet_group = int(total_per_interval[total_per_interval.tetrapod_group == tet_group].individuals_total)
	for seed in range(1, 11, 1):
		print("Starting {} : {}".format(tet_group, seed))
		sim = os.path.join(output_dir, "data_{}_{}.db".format(job_type, seed))
		# print("data_{}_{}.db".format(job_type, seed))
		tree = CoalescenceTree(sim, logging_level=40)
		# Drop existing tables
		tree._check_database()
		tree.cursor.execute("DROP TABLE IF EXISTS FRAGMENT_RICHNESS")
		tree.cursor.execute("DROP TABLE IF EXISTS BIODIVERSITY_METRICS")
		tree.cursor.execute("DROP TABLE IF EXISTS ALPHA_DIVERSITY")
		tree.cursor.execute("DROP TABLE IF EXISTS BETA_DIVERSITY")
		tree.database.commit()
		tmp_richness = []
		# Loop over the speciation parameters (including the potential metacommunities)
		for reference in tree.get_community_references():
			params = tree.get_community_parameters(reference)
			metacommunity_reference = params["metacommunity_reference"]
			if metacommunity_reference == 0:
				meta_params = {"metacommunity_size" : 0, "speciation_rate" : 0.0}
			else:
				meta_params = tree.get_metacommunity_parameters(metacommunity_reference)
			sad = tree.get_species_abundances(reference=reference)
			individuals = []
			for species_id, abundance in sad:
				individuals.extend([species_id] * abundance)
			# Shuffle the list, repeated 10 times to randomly draw species richness values.
			tmp_tmp_richness = []
			all_gammas = []
			# print("Shuffling species abundances")
			for i in range(10):
				random.shuffle(individuals)
				index = 0
				for interval in intervals:
					interval_group_fragments = info_per_pcoord[(info_per_pcoord["interval"] == interval) & (info_per_pcoord["tetrapod_group"] == tet_group)]
					tmp_inds = 0
					for _, row in interval_group_fragments.iterrows():
						fragment_name = row["fragment_name"]
						no_inds = row["individuals_total"]
						tmp_inds += no_inds
						fragment_individuals = []
						fragment_individuals.extend(individuals[index:index+no_inds])
						index += no_inds
						rich = len(set(fragment_individuals))
						tmp_tmp_richness.append({"interval" : interval, "tetrapod_group" : tet_group, "iteration": i,
												"fragment" : fragment_name, "richness" : rich})
					all_gammas.append({"interval" : interval, "gamma" : len(set(individuals[index-tmp_inds:index]))})
			for interval in intervals:
				interval_group_fragments = info_per_pcoord[(info_per_pcoord["interval"] == interval) & (info_per_pcoord["tetrapod_group"] == tet_group)]
				interval_sel = [x for x in tmp_tmp_richness if x["interval"] == interval]
				interval_richness = []
				for _, row in interval_group_fragments.iterrows():
					fragment_name = row["fragment_name"]
					sel = [x["richness"] for x in interval_sel if x["fragment"] == fragment_name]
					mean_richness = sum(sel)/len(sel)
					interval_richness.append({"seed" : seed, "reference" : reference, "speciation_rate" : params["speciation_rate"],
					                          "metacommunity_size" : meta_params["metacommunity_size"],
					                          "metacommunity_speciation_rate": meta_params["speciation_rate"],
					                          "interval" : interval, "tetrapod_group" : tet_group,
					                          "fragment" : fragment_name, "richness" : mean_richness})
				
				r = [x["richness"] for x in interval_richness]
				interval_gammas = [x["gamma"] for x in all_gammas if x["interval"] == interval]
				gamma = sum(interval_gammas)/len(interval_gammas)
				alpha = sum(r)/len(r)
				beta = gamma/alpha
				output_alpha.append({"interval" : interval, "tetrapod_group" : tet_group,
								 "seed" : seed, "reference" : reference,
								 "speciation_rate" : params["speciation_rate"],
								 "metacommunity_size" : meta_params["metacommunity_size"],
								 "metacommunity_speciation_rate": meta_params["speciation_rate"],
								 "alpha" : alpha, "beta" : beta, "species_richness" : gamma})
				tmp_richness.extend(interval_richness)
		output_richness.extend(tmp_richness)

Starting amphibian : 1


Starting amphibian : 2


Starting amphibian : 3


Starting amphibian : 4


Starting amphibian : 5


...

Starting amniote : 6


Starting amniote : 7


Starting amniote : 8


Starting amniote : 9


Starting amniote : 10


In [10]:
output_alpha_df = pd.DataFrame(output_alpha)
output_alpha_df.to_csv(os.path.join(output_csv_directory, "nse_metrics1.csv"))

In [ ]:
output_fragment_richness_df = pd.DataFrame(output_richness)
# This is to reduce the file size
output_fragment_richness_df = output_fragment_richness_df[output_fragment_richness_df.metacommunity_size > 0]
output_fragment_richness_df.to_csv(os.path.join(output_csv_directory, "nse_fragment_richness1.csv"))

In [ ]:
print("Test")